In [1]:
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
import keras

# Linear models in Keras

In [3]:
x = random((30, 2))
y = np.dot(x, [2., 3.]) + 1

https://keras.io/getting-started/sequential-model-guide/
- The sequential model is a linear stack of layers
- You can create a sequential model by passing a list of layer instances to the constructor
- you can also simply add layers via the .add() method
- The model needs to know what input shape it should expect. For this reason, the first layer in a sequential model needs to receive information about its input shape.
- And only the first, because following layers can do automatic shape inference

keras.layers.Dense : https://keras.io/layers/core/#dense

Just your regular densely-connected NN layer

In [5]:
keras_lm_model = keras.models.Sequential([
    keras.layers.Dense(1, input_shape = (2,))
])

Before training a model, you need to configure the learning process, which is done via the compile method. It receives three arguments
1. optimizer
2. loss function
3. list of metrics

In [6]:
keras_lm_model.compile(optimizer = keras.optimizers.SGD(lr = 0.1),
                      loss = 'mse')

keras_lm_model.evaluate: Computes the loss on some input data, batch by batch

In [8]:
keras_lm_model.evaluate(x, y, verbose = 0)

22.603702545166016

In [9]:
keras_lm_model.fit(x, y, nb_epoch = 20, batch_size = 1, verbose = 2)

Epoch 1/20
0s - loss: 1.8421
Epoch 2/20
0s - loss: 0.3012
Epoch 3/20
0s - loss: 0.1916
Epoch 4/20
0s - loss: 0.1031
Epoch 5/20
0s - loss: 0.0606
Epoch 6/20
0s - loss: 0.0313
Epoch 7/20
0s - loss: 0.0219
Epoch 8/20
0s - loss: 0.0116
Epoch 9/20
0s - loss: 0.0073
Epoch 10/20
0s - loss: 0.0039
Epoch 11/20
0s - loss: 0.0021
Epoch 12/20
0s - loss: 0.0015
Epoch 13/20
0s - loss: 7.5785e-04
Epoch 14/20
0s - loss: 4.2322e-04
Epoch 15/20
0s - loss: 2.5516e-04
Epoch 16/20
0s - loss: 1.5537e-04
Epoch 17/20
0s - loss: 8.3738e-05
Epoch 18/20
0s - loss: 4.5479e-05
Epoch 19/20
0s - loss: 2.6993e-05
Epoch 20/20
0s - loss: 1.4263e-05


In [10]:
keras_lm_model.evaluate(x, y, verbose = 0)

1.3549431969295256e-05

In [11]:
keras_lm_model.get_weights()

[array([[ 1.9905],
        [ 2.9908]], dtype=float32), array([ 1.0082], dtype=float32)]

# remove vgg's last layer and add our own linear layer

In [12]:
from vgg16 import Vgg16

In [13]:
vgg = Vgg16()

In [14]:
model = vgg.model

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [16]:
# removing the last layer
model.pop()

In [17]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [18]:
# making all the layers as non-trainable
for layer in model.layers:
    layer.trainable = False

In [19]:
# add a new dense layer
model.add(
    keras.layers.Dense(2, activation = 'softmax')
)

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [21]:
# create batch objects
gen = image.ImageDataGenerator()

In [26]:
??get_batches

In [22]:
train_batches = gen.flow_from_directory(
    directory = './data/redux/train/',
    target_size = (224, 224),
    shuffle = True,
    batch_size = 12,
    class_mode='categorical'
)

Found 23000 images belonging to 2 classes.


In [23]:
valid_batches = gen.flow_from_directory(
    directory = './data/redux/valid/',
    target_size = (224, 224),
    shuffle = True,
    batch_size = 12,
    class_mode='categorical'
)

Found 2000 images belonging to 2 classes.


In [ ]:
??model.fit

In [ ]:
??model.fit_generator

In [24]:
model.compile(
    optimizer = keras.optimizers.RMSprop(lr = 0.1),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [25]:
train_batches.n

23000

In [26]:
model.fit_generator(
    generator = train_batches,
    samples_per_epoch = train_batches.n,
    nb_epoch = 1,
    validation_data = valid_batches,
    nb_val_samples = valid_batches.n,
    verbose = 2
)

Epoch 1/1
629s - loss: 0.6165 - acc: 0.9607 - val_loss: 0.4597 - val_acc: 0.9700


In [ ]:
??vgg.test

In [27]:
non_shuffled_valid_batches = gen.flow_from_directory(
    directory = './data/redux/valid/',
    target_size = (224, 224),
    shuffle = False,
    batch_size = 12,
    class_mode='categorical'
)

Found 2000 images belonging to 2 classes.


In [28]:
# get predicted probabilities
predicted_probs = model.predict_generator(non_shuffled_valid_batches, non_shuffled_valid_batches.n)

In [29]:
predicted_probs[:8]

array([[  4.2931e-16,   1.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00]], dtype=float32)

In [30]:
# plotting few images
plots([
    keras.preprocessing.image.load_img(non_shuffled_valid_batches.directory + non_shuffled_valid_batches.filenames[0]),
    keras.preprocessing.image.load_img(non_shuffled_valid_batches.directory + non_shuffled_valid_batches.filenames[1])
],
    titles = predicted_probs[:2, 1])

In [31]:
predicted_probs

array([[  4.2931e-16,   1.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       ..., 
       [  0.0000e+00,   1.0000e+00],
       [  0.0000e+00,   1.0000e+00],
       [  0.0000e+00,   1.0000e+00]], dtype=float32)